In [40]:
using RigidBodyDynamics
using GeometryTypes
using Interact
import RigidBodyTreeInspector
import DrakeVisualizer: Visualizer, draw
import DataStructures: OrderedDict

In [46]:
include("arms.jl")
include("depth_sensors.jl")

DepthSensors

In [47]:
function beanbag()
    limbs = OrderedDict{RigidBody, Arms.Limb}()
    
    mechanism = Mechanism{Float64}("world")
    parent = root_body(mechanism)
    
    joint = Joint("joint1", QuaternionFloating())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    surface_points = Vector{Point3D}()
    skeleton_points = [Point3D(body.frame, Vec(0.,0,0))]
    for axis = 1:3
        for s = [-1; 1]
            x = [0.; 0; 0]
            x[axis] = s
            push!(surface_points, Point3D(body.frame, Vec(x)))
        end
    end
    limbs[body] = Arms.Limb(surface_points, skeleton_points)
    
    Arms.Model(mechanism, limbs)
end

beanbag (generic function with 1 method)

In [48]:
b = beanbag()

Arms.Model(Vertex: world (root)
  Vertex: body1, Edge: joint1,DataStructures.OrderedDict{RigidBodyDynamics.RigidBody{T<:Real},Arms.Limb}(RigidBody: "body1"=>Arms.Limb(RigidBodyDynamics.Point3D[Point3D in "body1": Vec(-1.0,0.0,0.0),Point3D in "body1": Vec(1.0,0.0,0.0),Point3D in "body1": Vec(0.0,-1.0,0.0),Point3D in "body1": Vec(0.0,1.0,0.0),Point3D in "body1": Vec(0.0,0.0,-1.0),Point3D in "body1": Vec(0.0,0.0,1.0)],RigidBodyDynamics.Point3D[Point3D in "body1": Vec(0.0,0.0,0.0)])))

In [49]:
state = Arms.ModelState(b)
draw(b, state)

In [53]:
joint_angles, deformations = Arms.zero_configuration(b)
state = Arms.ModelState(b, joint_angles, deformations)
@manipulate for x = linspace(-1, 1, 51)
    state.limb_deformations[1][1] = FreeVector3D(b.mechanism.toposortedTree[2].vertexData.frame, Vec(x, 0, 0))
    draw(b, state)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),true)

nothing